**This Notebook is by Soumyadip Sarkar and this version is simply to help him get going by using preprocessed data and fixing a few things.**

It is not a complete solution and it still has issues that I commented out and I will leave it to him to work through.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#!pip install -U pandas_bokeh

In [3]:
import cv2
import gc
import os

import numpy as np 
import pandas as pd 

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
# import tensorflow.keras.applications.ResNet101 as resnet101

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
%matplotlib inline

# import pandas_bokeh
# from bokeh.models.widgets import DataTable, TableColumn
# from bokeh.models import ColumnDataSource

from plotly.offline import iplot
import plotly as py
import plotly.tools as tls
import cufflinks as cf

py.offline.init_notebook_mode(connected = True)
cf.go_offline()
cf.set_config_file(theme = 'solar')

# pd.set_option('plotting.backend', 'pandas_bokeh')
# pandas_bokeh.output_notebook()

In [4]:
print(tf.__version__)

2.2.0


In [5]:
SEED = 42
EPOCH = 10
BATCH_SIZE = 32 #increased for small images taking less memory
IMG_SIZE = 224
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [6]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv',na_values=['unknown'])
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
submission = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')

train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,NaN,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,NaN,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,NaN,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,NaN,benign,0


In [7]:
gc.collect()

24

In [8]:
#DIR = '../input/siim-isic-melanoma-classification/jpeg/train/'
DIR = '../input/siic-isic-224x224-images/train/'
img = []
labels = []
#jpg = '.jpg'
png = '.png'

for i in train['image_name']:
    img.append(os.path.join(DIR,i)+png)
    
for i in train['target']:
    labels.append(i)
    

In [9]:
x_train,x_val,y_train,y_val = train_test_split(img,labels,test_size = 0.2,random_state = SEED)

In [10]:
train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                                         rescale=1./255,
                                        rotation_range=40,
                                         horizontal_flip=True,
                                         vertical_flip= True,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        
)

val_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                                            rescale=1./255
)

In [11]:
train_img = pd.DataFrame(x_train,columns=['image'])
train_labels = pd.DataFrame(y_train,columns=['target'])
train_data = pd.concat([train_img,train_labels],axis = 1)

val_img = pd.DataFrame(x_val,columns=['image'])
val_labels = pd.DataFrame(y_val,columns=['target'])
val_data = pd.concat([val_img,val_labels],axis = 1)

train_data.head()

,image,target
0,../input/siic-isic-224x224-images/train/ISIC_6...,0
1,../input/siic-isic-224x224-images/train/ISIC_6...,0
2,../input/siic-isic-224x224-images/train/ISIC_8...,0
3,../input/siic-isic-224x224-images/train/ISIC_4...,0
4,../input/siic-isic-224x224-images/train/ISIC_0...,0


In [12]:
train_img_gen = train_data_generator.flow_from_dataframe(train_data,
    x_col='image',
    y_col='target',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw')

val_img_gen = val_data_generator.flow_from_dataframe(val_data,
                                                    x_col = 'image',
                                                    y_col = 'target',
                                                    target_size= (IMG_SIZE,IMG_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    class_mode='raw')


Found 26500 validated image filenames.
Found 6626 validated image filenames.


In [13]:
# # , because of class imbalance it's better to use focal loss rather than normal binary_crossentropy.You can read more about it here

# def focal_loss(alpha=0.25,gamma=2.0):
#     def focal_crossentropy(y_true, y_pred):
#         bce = K.binary_crossentropy(y_true, y_pred)
        
#         y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
#         p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
#         alpha_factor = 1
#         modulating_factor = 1

#         alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
#         modulating_factor = K.pow((1-p_t), gamma)

#         # compute the final loss and return
#         return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
#     return focal_crossentropy

In [14]:
from tensorflow.python.keras import backend as K

def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [15]:
from tensorflow.keras import layers, models

model = models.Sequential()
model.add(tf.keras.applications.ResNet101(weights='imagenet',
                                        include_top=False,
                                        input_shape=(IMG_SIZE,IMG_SIZE,3)
                                       ))
model.add(layers.GlobalAveragePooling2D())
#model.add(layers.Flatten())
#model.add(layers.BatchNormalization())
#model.add(layers.Dense(1024,activation= 'relu'))
#model.add(layers.BatchNormalization())
# model.add(layers.Dense(70000,activation= 'relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dense(20000,activation= 'relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dense(1000,activation= 'relu'))
# model.add(layers.BatchNormalization())
#model.add(layers.Dense(256,activation= 'relu'))
#model.add(layers.BatchNormalization())
model.add(layers.Dense(1,activation='sigmoid'))
model.layers[0].trainable = True

model.compile(loss=[focal_loss(alpha=0.25,gamma=2.0)],metrics=[tf.keras.metrics.AUC()],optimizer='adam' )

171450368/171446536 [==============================] - 1s 0us/step


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Model)            (None, 7, 7, 2048)        42658176  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 42,660,225
Trainable params: 42,554,881
Non-trainable params: 105,344
_________________________________________________________________


In [17]:
# from tf.keras.callbacks import ModelCheckpoint
# filepath = "../output/saved_models-improvement-{epoch:02d}-{auc:.2f}.hdf5"
# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,monitor = 'auc',verbose = 1,save_best_only = True,mode = 'max')
# callbacks_list = [checkpoint]

mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5',monitor=tf.keras.metrics.AUC(),mode='min',save_best_only=True,verbose=1)

In [18]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_data.target),
                                                 train_data.target)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass classes=[0 1], y=0        0
1        0
2        0
3        0
4        0
        ..
26495    0
26496    0
26497    0
26498    0
26499    0
Name: target, Length: 26500, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error



In [19]:
class_weight ={
    0:0.50893029,
    1:28.49462366
}
print(class_weight)

{0: 0.50893029, 1: 28.49462366}


In [20]:
gc.collect()

80

In [21]:

History = model.fit_generator(train_img_gen,
                             steps_per_epoch=train_data.shape[0]//BATCH_SIZE,
                             epochs=EPOCH,
                             validation_data=val_img_gen,
                             validation_steps=val_data.shape[0]//BATCH_SIZE,
                             class_weight=class_weight,
                             callbacks=[mc]
                             )

Epoch 1/10
828/828 [==============================] - 536s 648ms/step - loss: 0.1789 - auc: 0.5280 - val_loss: 7.4103 - val_auc: 0.3466
Epoch 2/10
828/828 [==============================] - 480s 580ms/step - loss: 0.1200 - auc: 0.6311 - val_loss: 0.0514 - val_auc: 0.7182
Epoch 3/10
828/828 [==============================] - 480s 579ms/step - loss: 0.0784 - auc: 0.7123 - val_loss: 2.9083 - val_auc: 0.6028
Epoch 4/10
828/828 [==============================] - 482s 582ms/step - loss: 0.0758 - auc: 0.7046 - val_loss: 0.0259 - val_auc: 0.7493
Epoch 5/10
828/828 [==============================] - 486s 587ms/step - loss: 0.0664 - auc: 0.7232 - val_loss: 0.0554 - val_auc: 0.7543
Epoch 6/10
828/828 [==============================] - 488s 590ms/step - loss: 0.0673 - auc: 0.7224 - val_loss: 0.0370 - val_auc: 0.7511
Epoch 7/10
828/828 [==============================] - 494s 596ms/step - loss: 0.1001 - auc: 0.6774 - val_loss: 0.0753 - val_auc: 0.5111
Epoch 8/10
828/828 [============================

In [22]:
tf.keras.models.save_model(model,'model_resnet101.h5') 

In [23]:
model.save('my_resnet101.h5')

Here I useed the Resnet101 net work, with Image Aug. And as the data set is imbalanced so I used focal loss and weighted class approach.

# Thank you for reading!!!

resnet


In [24]:
# loss, accu = model.evaluate(X_val, Y_val)

In [25]:
# test_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
# test_data=[]
# for i in range(test.shape[0]):
#     test_data.append(test_dir + test['image_name'].iloc[i]+'.jpg')
# df_test=pd.DataFrame(test_data)
# df_test.columns=['images']

In [26]:
# target=[]
# for path in df_test['images']:
#     img=cv2.imread(str(path))
#     img = cv2.resize(img, (224,224))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = img.astype(np.float32)/255.
#     img=np.reshape(img,(1,224,224,3))
#     prediction=model.predict(img)
#     target.append(prediction[0][0])

# # submission['target']=target

In [27]:
# submission['target']=target

In [28]:
# submission.to_csv('submission.csv', index=False)
# submission.head()

i am a student